In [12]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Market data
S = 30        # Spot price
K = 27        # Strike price
r = 0.04      # Risk-free rate
q = 0.01      # Dividend yield
T = 7 / 12    # Time to maturity
market_price = 4.5
tol = 1e-6
max_iter = 100

# Black-Scholes Call Price
def bs_call_price(sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Vega (derivative of BS call price with respect to sigma)
def vega(sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

# Newton-Raphson Method
def newton_method(sigma0):
    sigma = sigma0
    records = []
    for i in range(max_iter):
        price = bs_call_price(sigma)
        v = vega(sigma)
        error = price - market_price
        sigma_new = sigma - error / v
        records.append((i, sigma, price, error, v, sigma_new))
        if abs(error) < tol:
            break
        sigma = sigma_new
    df_newton = pd.DataFrame(records, columns=["iter", "sigma", "price", "error", "vega", "next_sigma"])
    return df_newton

# Secant Method
def secant_method(sigma0, sigma1):
    f0 = bs_call_price(sigma0) - market_price
    f1 = bs_call_price(sigma1) - market_price
    records = [(0, sigma0, f0, None, sigma1)]
    for i in range(1, max_iter):
        slope = (f1 - f0) / (sigma1 - sigma0)
        sigma_new = sigma1 - f1 / slope
        f_new = bs_call_price(sigma_new) - market_price
        records.append((i, sigma1, f1, slope, sigma_new))
        if abs(f_new) < tol:
            break
        sigma0, f0 = sigma1, f1
        sigma1, f1 = sigma_new, f_new
    df_secant = pd.DataFrame(records, columns=["iter", "sigma", "error", "slope", "next_sigma"])
    return df_secant

# Run both methods
df_newton = newton_method(0.5)
df_secant = secant_method(0.5, 0.501)



In [16]:
print(df_newton.to_latex(caption="Newton-Raphson Method Results", index=False, escape=True, label="tab:newton_results"))

\begin{table}
\caption{Newton-Raphson Method Results}
\label{tab:newton_results}
\begin{tabular}{rrrrrr}
\toprule
iter & sigma & price & error & vega & next\_sigma \\
\midrule
0 & 0.500000 & 6.194813 & 1.694813 & 7.968646 & 0.287315 \\
1 & 0.287315 & 4.552267 & 0.052267 & 7.262646 & 0.280118 \\
2 & 0.280118 & 4.500202 & 0.000202 & 7.205791 & 0.280090 \\
3 & 0.280090 & 4.500000 & 0.000000 & 7.205562 & 0.280090 \\
\bottomrule
\end{tabular}
\end{table}



In [17]:
print(df_secant.to_latex(caption="Secant Method Results", index=False, escape=True, label="tab:secant_results"))

\begin{table}
\caption{Secant Method Results}
\label{tab:secant_results}
\begin{tabular}{rrrrr}
\toprule
iter & sigma & error & slope & next\_sigma \\
\midrule
0 & 0.500000 & 1.694813 & NaN & 0.501000 \\
1 & 0.501000 & 1.702782 & 7.969178 & 0.287329 \\
2 & 0.287329 & 0.052370 & 7.724082 & 0.280549 \\
3 & 0.280549 & 0.003307 & 7.236335 & 0.280092 \\
4 & 0.280092 & 0.000013 & 7.207446 & 0.280090 \\
\bottomrule
\end{tabular}
\end{table}

